# Classification using Cyclic Boosting

First, install the  package and its dependencies

```sh
!pip install cyclic-boosting
```

In [1]:
import pandas as pd
import numpy as np

Let's load the test dataset from Blue-Yonder-OSS

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("./archive_2/Walmart.csv")
df["Date"] = pd.to_datetime(df["Date"], format='%d-%m-%Y')
df.to_csv("./Walmart.csv", index=False)

In [3]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106


# Automated Machine Learning with Tornado
With tornado, you can automatically perform data preparation, feature property setting, hyperparameter tuning, model building, training, evaluation, and plotting!

In [4]:
from cyclic_boosting.tornado import Generator, Manager, Trainer

data_deliverler = Generator.TornadoDataModule("./Walmart.csv")
manager = Manager.TornadoVariableSelectionModule()
trainer = Trainer.SqueezeTrainer(data_deliverler, manager)
trainer.run(target="weekly_sales", log_policy="compute_COD", verbose=False)

Auto analysis target ['temperature', 'fuel_price', 'cpi', 'unemployment']
    has_trend: ['temperature', 'fuel_price', 'cpi', 'unemployment']
    has_seasonality: ['temperature', 'fuel_price']
    has_up_monotonicity: []
    has_down_monotonicity: []
    has_linearity: []
    has_missing: []


/home/tsuruda_yoshito/work/tornado/rPrivate-cyclic-boosting/.venv/lib/python3.10/site-packages/statsmodels/tsa/stl/mstl.py:217: UserWarning: A period(s) is larger than half the length of time series. Removing these period(s).
  warnings.warn(
iter: 5 / 35 Encountered negative change of loss. This might not be a problem, as long as the model converges. Check the LOSS changes in the analysis plots.
iter: 35 / 35 
TRUNCATED
['store', 'unemployment', 'cpi', 'dayofyear', 'temperature', 'holiday_flag', 'dayofweek', 'fuel_price', ('store', 'cpi'), ('store', 'unemployment'), ('store', 'dayofweek'), ('store', 'holiday_flag'), ('unemployment', 'dayofweek'), ('holiday_flag', 'unemployment'), ('temperature', 'dayofweek'), ('cpi', 'dayofweek'), ('holiday_flag', 'cpi'), ('dayofweek', 'dayofyear'), ('holiday_flag', 'dayofyear'), ('fuel_price', 'dayofweek'), ('temperature', 'fuel_price'), ('cpi', 'dayofyear'), ('temperature', 'dayofyear'), ('unemployment', 'dayofyear'), ('cpi', 'unemployment'), ('fuel

# Load the best model and make predictions.

Get the best model path.

In [5]:
import pickle
from pathlib import Path

model_nos = []
for p in sorted(Path("./models/").glob("model*")):
    model_nos.append(str(p)[str(p).find("_") + 1 :])
model_path = f"./models/model_{model_nos[-1]}/model_{model_nos[-1]}.pkl"
print(model_path)

./models/model_20/model_20.pkl


Make predictions with the best model.

In [6]:
data = {
    "dayofweek": [4],
    "dayofyear": [190],
    "event": [0],
    "l_id": [1],
    "normal_price": [10.34],
    "p_id": [20],
    "pg_id_1": [1],
    "pg_id_2": [1],
    "pg_id_3": [1],
    "promotion_type": [1],
    "sales_area": [6321.6800893695445],
    "sales_price": [10.34],
    "school_holiday": [0],
}

X = pd.DataFrame(data)

with open(model_path, "rb") as f:
    CB_est = pickle.load(f)
    yhat = CB_est.predict(X.copy())
    print(yhat)

ValueError: Input Matrix X has not the same number of feature columns (2) as the matrix in the fit (8).